# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5,min_tracking_confidence=0.2, max_num_hands = 2))

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

# 3. Extract Keypoint Values

In [9]:
def extract_keypoints(results):
    data_aux = []

    x_ = []
    y_ = []

    if results.multi_hand_landmarks:
        cnt = 0
        for hand_landmarks in results.multi_hand_landmarks:
            cnt += 1
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))
                
        if (cnt == 1):
            for i in range(21, 42):
                data_aux.append(0.0)
                data_aux.append(0.0)   
    else:
        for i in range(84):
            data_aux.append(0.0)                
    return np.array(data_aux)                

In [13]:
DATA_PATH = os.path.join('./10_day10_mousa/') 
DATA_PATH_IMAGES = os.path.join('./mousa_data_images/') 

#  'Violet', 'Grey', 'Pink', 'Blue'
# Actions that we try to detect
actions = np.array(['BB'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 31

In [14]:
for action in actions: 
    for sequence in range(start_folder, start_folder + 30):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            os.makedirs(os.path.join(DATA_PATH_IMAGES, action, str(sequence)))
        except:
            pass

In [15]:
for action in actions: 
    for sequence in range(start_folder + 150, start_folder + 180):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            os.makedirs(os.path.join(DATA_PATH_IMAGES, action, str(sequence)))
        except:
            pass

In [16]:
# url = "http://192.168.1.10:8080/video"  # Example

# cap = cv2.VideoCapture(url)

cap = cv2.VideoCapture(0)
# Set mediapipe model     
# NEW LOOP
# Loop through actions
for action in actions:
    print(f"next word {action} after 5 seconds...")
    cv2.waitKey(5000)
    
    # Loop through sequences aka videos
    for sequence in range(start_folder, start_folder+no_sequences):
        print(sequence)
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):
            

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, hands)
            

            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
            
            # NEW Export keypoints
            flipped_img = cv2.flip(frame, 1)
            flipped_img, flipped_results = mediapipe_detection(flipped_img, hands)
            
            keypoints = extract_keypoints(results)
            flipped_keypoints = extract_keypoints(flipped_results)
            
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num + 1))
            npy_path_flipped = os.path.join(DATA_PATH, action, str(sequence + 150), str(frame_num + 1))
            
            img_path = os.path.join(DATA_PATH_IMAGES, action, str(sequence), str(frame_num + 1) + '.jpg')
            img_path_flipped = os.path.join(DATA_PATH_IMAGES, action, str(sequence + 150), str(frame_num + 1) + '.jpg')
            
            
            np.save(npy_path, keypoints)
            np.save(npy_path_flipped, flipped_keypoints)
            cv2.imwrite(img_path, frame)
            cv2.imwrite(img_path_flipped, flipped_img)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
cap.release()
cv2.destroyAllWindows()

next word BB after 5 seconds...
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [16]:
frame = np.load("C:/Users/Mousa/Desktop/created_data/10_day10_mousa/BB/31/5.npy")
frame

array([0.16838574, 0.14938569, 0.0906648 , 0.11687475, 0.05338949,
       0.06938493, 0.03025353, 0.03458309, 0.        , 0.00661373,
       0.14407444, 0.01885056, 0.06256139, 0.        , 0.04435378,
       0.03188556, 0.05630559, 0.05202192, 0.18100423, 0.0269385 ,
       0.07376504, 0.02044183, 0.06548613, 0.05597949, 0.0879873 ,
       0.06913716, 0.20398039, 0.04388231, 0.09296554, 0.05146927,
       0.09047967, 0.08227056, 0.11777502, 0.0873853 , 0.21108949,
       0.06628579, 0.12103665, 0.07664847, 0.11380488, 0.09656823,
       0.13622522, 0.09860635, 0.00740171, 0.15155679, 0.08178645,
       0.12975013, 0.11264614, 0.0933305 , 0.14684511, 0.06676477,
       0.17905418, 0.04190373, 0.03451622, 0.0404042 , 0.08749286,
       0.        , 0.1485594 , 0.01061481, 0.17609643, 0.03016752,
       0.00679768, 0.04451931, 0.10502837, 0.01054168, 0.16083377,
       0.03921783, 0.1651727 , 0.06133479, 0.        , 0.05946243,
       0.10486872, 0.03588212, 0.14552515, 0.06937695, 0.13366